In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from thefuzz import fuzz
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from gensim.models import Word2Vec
import streamlit as st

# Load SpaCy English model
import subprocess
import sys
try:
    nlp = spacy.load('en_core_web_sm')
except:
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load('en_core_web_sm')

# Helper function for Word2Vec
def prepare_sentences(text_list):
    """Prepare text for Word2Vec training"""
    return [text if isinstance(text, list) else text.split() for text in text_list]

# Complete Job Postings Data
jobs_data = [
    {
        'title': 'Senior WordPress Developer',
        'description': '''We are looking for a skilled WordPress Website Developer with expertise in end-to-end development, 
        deployment, and maintenance of dynamic websites. Join our innovative team and contribute to creating outstanding digital experiences.''',
        'requirements': '''
            WordPress PHP HTML5 CSS3 JavaScript MySQL Git DevOps server setup hosting DNS 
            React Vue.js Angular version control CI/CD SEO accessibility cloud services AWS Google Cloud Azure
            custom theme plugin development server setup hosting environments DNS configuration
            performance optimization security debugging project management team collaboration
        ''',
        'location': 'Greater Noida',
        'salary_min': 500000,
        'salary_max': 600000,
        'education': 'B.tech',
        'experience_min': 1
    },
    {
        'title': 'Graphic Designer',
        'description': 'Create visually appealing graphics for various platforms including websites, social media and print.',
        'requirements': '''
            Figma Adobe Illustrator Adobe Photoshop InDesign web design HTML CSS 
            team collaboration visual design UI/UX design typography color theory
            brand guidelines layout design digital marketing social media design
            responsive design creative thinking problem-solving communication
        ''',
        'location': 'Noida',
        'salary_min': 200000,
        'salary_max': 300000,
        'education': 'Any',
        'experience_min': 1
    },
    {
        'title': 'Cinematographer',
        'description': 'Create high-quality video content for our streaming platform.',
        'requirements': '''
            video production direction cinematography Adobe Premiere Pro
            photography Adobe Photoshop video editing lighting composition
            storyboarding shot planning camera operation sound recording
            color grading post-production workflow management equipment handling
        ''',
        'location': 'Any',
        'salary_min': 200000,
        'salary_max': 300000,
        'education': 'Any',
        'experience_min': 0
    },
    {
        'title': 'Financial Sales Officer',
        'description': 'Handle education loan processing and customer relations.',
        'requirements': '''
            banking finance sales customer service loan processing
            documentation communication skills relationship management
            MS Office financial analysis customer acquisition negotiation
            business development market research presentation skills
        ''',
        'location': 'Hyderabad',
        'salary_min': 400000,
        'salary_max': 430000,
        'education': 'Graduate',
        'experience_min': 0
    },
    {
        'title': 'Associate Content Writer',
        'description': 'Create engaging content for various digital platforms.',
        'requirements': '''
            content writing blogging SEO social media marketing
            copywriting editing proofreading research skills
            digital marketing content strategy keyword research
            WordPress CMS English proficiency creative writing
        ''',
        'location': 'Any',
        'salary_min': 200000,
        'salary_max': 400000,
        'education': 'Any',
        'experience_min': 0
    }
]

# Complete Candidate Profiles Data
candidates_data = [
    {
        'name': 'PERSON1',
        'resume': '''
            Technical Skills
            Languages: Python (NumPy, Pandas), PHP 7+, Bash, Java, JavaScript (ES6+), C++, MATLAB
            Frontend: React.js, HTML5/CSS3
            Backend: Express.js, Node.js, Flask, PHP
            DevOps & Tools: Git, Docker, Apache, Nginx, PyCharm
            Databases: MySQL, PostgreSQL, MongoDB, SQLite
            Operating Systems: Linux, MacOS, Windows
            Education: B.Tech
            Experience: 1 year
            Location: Mangalore
            Expected Salary: 1000000
        '''
    },
    {
        'name': 'PERSON2',
        'resume': '''
            Senior WordPress Developer
            Experience: 19 years
            Skills: WordPress PHP HTML CSS JavaScript web development cross-browser compatibility 
            QA testing project management theme customization plugin development
            Education: 12th Pass
            Location: San Francisco
            Expected Salary: 400000
        '''
    },
    {
        'name': 'PERSON3',
        'resume': '''
            WordPress Developer
            Skills: PHP HTML JavaScript WordPress development technical integration
            cross-browser compatibility QA testing design marketing
            WordPress sites architecture design development SEO SEM
            Education: Bachelor's Degree in Computer Science
            Experience: 10 years
            Location: San Francisco
            Expected Salary: 600000
        '''
    },
    {
        'name': 'PERSON4',
        'resume': '''
            Senior Graphic Designer
            Skills: Adobe Creative Suite Sketch Figma InVision Adobe XD
            HTML5 CSS3 JavaScript WordPress Bootstrap Webflow Typography
            Color Theory Logo Design Brand Guidelines Visual Storytelling
            Education: Master of Fine Arts - Graphic Design
            Experience: 11 years
            Location: Any
            Expected Salary: 500000
        '''
    },
    {
        'name': 'PERSON5',
        'resume': '''
            Lead Graphic Designer
            Skills: Figma Adobe Creative Suite UI/UX Design HTML5 CSS3
            JavaScript WordPress Bootstrap Webflow Branding Visual Identity
            Typography Color Theory Logo Design Brand Guidelines
            Education: Master of Fine Arts - Graphic Design
            Experience: 13 years
            Location: Any
            Expected Salary: 600000
        '''
    }
]

# Preprocessing Functions
def preprocess_text(text):
    """Preprocess text using spaCy"""
    try:
        # Convert to string and lowercase
        text = str(text).lower()
        
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = ' '.join(text.split())
        
        # Process with spaCy
        doc = nlp(text)
        
        # Get lemmatized tokens, excluding stopwords
        tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
        
        return tokens
    except Exception as e:
        print(f"Error in preprocessing text: {str(e)}")
        return []

def extract_skills(text):
    """Extract skills from text using predefined patterns and spaCy"""
    skills = set()
    
    # Comprehensive skill patterns
    skill_patterns = {
        'programming_languages': [
            'python', 'java', 'javascript', 'php', 'c++', 'ruby', 'matlab',
            'typescript', 'scala', 'kotlin', 'swift', 'r', 'bash', 'golang',
            'rust', 'perl', 'shell scripting'
        ],
        'web_technologies': [
            'html', 'css', 'react', 'angular', 'vue', 'node', 'express',
            'django', 'flask', 'wordpress', 'bootstrap', 'jquery', 'sass',
            'less', 'webpack', 'gatsby', 'next.js', 'nuxt.js', 'svelte',
            'web components', 'pwa', 'responsive design'
        ],
        'databases': [
            'mysql', 'postgresql', 'mongodb', 'sqlite', 'oracle', 'sql server',
            'redis', 'elasticsearch', 'cassandra', 'dynamodb', 'mariadb',
            'neo4j', 'couchdb', 'firebase'
        ],
        'cloud_devops': [
            'aws', 'azure', 'gcp', 'docker', 'kubernetes', 'jenkins', 'git',
            'terraform', 'ansible', 'circleci', 'travis', 'gitlab ci',
            'github actions', 'cloud formation', 'puppet', 'chef'
        ],
        'design_tools': [
            'photoshop', 'illustrator', 'figma', 'sketch', 'indesign', 'xd',
            'premiere', 'after effects', 'lightroom', 'blender', 'maya',
            'zeplin', 'invision', 'principle'
        ],
        'soft_skills': [
            'communication', 'leadership', 'teamwork', 'problem solving',
            'project management', 'time management', 'analytical thinking',
            'creativity', 'collaboration', 'adaptability'
        ],
        'marketing_skills': [
            'seo', 'sem', 'content marketing', 'social media marketing',
            'email marketing', 'digital marketing', 'market research',
            'analytics', 'copywriting', 'brand management'
        ],
        'finance_skills': [
            'financial analysis', 'banking', 'investment', 'risk management',
            'portfolio management', 'trading', 'accounting', 'budgeting',
            'forecasting', 'financial modeling'
        ]
    }
    
    # Convert text to lowercase for matching
    text_lower = text.lower()
    
    # Extract skills using pattern matching
    for category, pattern_list in skill_patterns.items():
        for skill in pattern_list:
            if skill in text_lower:
                skills.add(skill)
    
    # Use spaCy for additional skill extraction
    doc = nlp(text)
    
    # Extract named entities that might be skills
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PRODUCT', 'GPE']:
            skills.add(ent.text.lower())
    
    # Extract noun phrases that might be skills
    for chunk in doc.noun_chunks:
        if len(chunk.text.split()) <= 3:  # Limit to phrases of 3 words or less
            skills.add(chunk.text.lower())
    
    return list(skills)

def extract_entities(text):
    """Extract various entities from text"""
    doc = nlp(text)
    
    entities = {
        'skills': extract_skills(text),
        'education': [],
        'experience': '0',
        'location': '',
        'expected_salary': '0'
    }
    
    # Comprehensive education patterns
    education_patterns = [
        r'b\.?tech', r'b\.?e', r'm\.?tech', r'm\.?e', r'mba', r'phd',
        r'bachelor[\'s]* degree', r'master[\'s]* degree', r'doctorate',
        r'diploma', r'certification', r'12th', r'high school',
        r'post graduate', r'postgraduate', r'graduate'
    ]
    
    # Extract education
    text_lower = text.lower()
    for pattern in education_patterns:
        if re.search(pattern, text_lower):
            match = re.search(pattern, text_lower).group()
            entities['education'].append(match)
    
    # Enhanced experience patterns
    experience_patterns = [
        r'(\d+)\s*(?:years?|yrs?)\s*(?:of\s*)?experience',
        r'experience\s*:\s*(\d+)\s*(?:years?|yrs?)',
        r'(\d+)\s*(?:years?|yrs?)\s*exp',
        r'worked\s*(?:for\s*)?(\d+)\s*(?:years?|yrs?)',
        r'(\d+)\+?\s*years?'
    ]
    
    # Extract experience
    max_experience = 0
    for pattern in experience_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            try:
                years = int(match.group(1))
                max_experience = max(max_experience, years)
            except:
                continue
    
    if max_experience > 0:
        entities['experience'] = str(max_experience)
    
    # Enhanced location patterns
    location_patterns = [
        r'location\s*:\s*([a-zA-Z\s,]+)',
        r'based\s+in\s+([a-zA-Z\s,]+)',
        r'located\s+in\s+([a-zA-Z\s,]+)',
        r'(?:city|region|country)\s*:\s*([a-zA-Z\s,]+)',
        r'(?:working|residing)\s+in\s+([a-zA-Z\s,]+)'
    ]
    
    # Extract location
    for pattern in location_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            entities['location'] = match.group(1).strip()
            break
    
    # If no location found through patterns, try spaCy
    if not entities['location']:
        for ent in doc.ents:
            if ent.label_ in ['GPE', 'LOC']:
                entities['location'] = ent.text
                break
    
    # Enhanced salary patterns
    salary_patterns = [
        r'expected\s*salary\s*:\s*(\d[\d,]*)',
        r'salary\s*:\s*(\d[\d,]*)',
        r'ctc\s*:\s*(\d[\d,]*)',
        r'compensation\s*:\s*(\d[\d,]*)',
        r'package\s*:\s*(\d[\d,]*)'
    ]
    
    # Extract salary
    for pattern in salary_patterns:
        match = re.search(pattern, text_lower)
        if match:
            # Remove commas and convert to number
            salary = match.group(1).replace(',', '')
            entities['expected_salary'] = salary
            break
    
    return entities

# Enhanced Matching Functions
def get_sentence_vector(tokens, model):
    """Get vector representation of tokens using Word2Vec model"""
    vectors = []
    for token in tokens:
        try:
            vectors.append(model.wv[token])
        except (KeyError, AttributeError):
            continue
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

def fuzzy_skill_match(job_skills, candidate_skills):
    """Calculate fuzzy matching score between job skills and candidate skills"""
    if not job_skills or not candidate_skills:
        return 0
    
    match_scores = []
    for job_skill in job_skills:
        skill_scores = [
            max(
                fuzz.ratio(job_skill, cand_skill),
                fuzz.partial_ratio(job_skill, cand_skill),
                fuzz.token_sort_ratio(job_skill, cand_skill)
            ) / 100
            for cand_skill in candidate_skills
        ]
        match_scores.append(max(skill_scores))
    
    return np.mean(match_scores)

def calculate_education_score(required_edu, candidate_edu):
    """Calculate education match score"""
    education_levels = {
        'phd': 5,
        'doctorate': 5,
        "master's degree": 4,
        'master': 4,
        "bachelor's degree": 3,
        'bachelor': 3,
        'b.tech': 3,
        'graduate': 3,
        'diploma': 2,
        '12th pass': 2,
        'high school': 2,
        'any': 1
    }
    
    required_level = 1  # default for 'any'
    candidate_level = 1  # default minimum
    
    # Get candidate's highest education level
    for edu in candidate_edu:
        edu_lower = edu.lower()
        for level, score in education_levels.items():
            if level in edu_lower:
                candidate_level = max(candidate_level, score)
    
    # Get required education level
    required_edu_lower = required_edu.lower()
    for level, score in education_levels.items():
        if level in required_edu_lower:
            required_level = score
            break
    
    # Calculate score
    if required_level <= candidate_level:
        return 1.0
    else:
        return candidate_level / required_level

def compute_match_scores(jobs_df, candidates_df, w2v_model):
    """Compute comprehensive match scores between jobs and candidates"""
    results = []
    
    # Define weights for different criteria
    weights = {
        'skills': 0.4,      # Technical skills match
        'education': 0.15,  # Education level match
        'experience': 0.15, # Experience level match
        'location': 0.15,   # Location match
        'salary': 0.15      # Salary expectations match
    }
    
    for _, job in jobs_df.iterrows():
        job_vector = get_sentence_vector(job['processed_requirements'], w2v_model)
        
        for _, candidate in candidates_df.iterrows():
            # Get candidate vector
            candidate_vector = get_sentence_vector(candidate['skills'], w2v_model)
            
            # Calculate skill match scores
            vector_similarity = cosine_similarity([job_vector], [candidate_vector])[0][0]
            fuzzy_match = fuzzy_skill_match(job['processed_requirements'], candidate['skills'])
            skill_score = (vector_similarity + fuzzy_match) / 2
            
            # Calculate education match
            education_score = calculate_education_score(job['education'], candidate['education'])
            
            # Calculate experience match
            try:
                candidate_exp = float(candidate['experience'])
                required_exp = float(job['experience_min'])
                experience_score = min(1.0, candidate_exp / max(1, required_exp))
            except (ValueError, TypeError):
                experience_score = 0.0
            
            # Calculate location match
            if job['location'].lower() == 'any' or candidate['location'].lower() == 'any':
                location_score = 1.0
            else:
                location_score = 1.0 if job['location'].lower() in candidate['location'].lower() else 0.0
            
            # Calculate salary match
            try:
                candidate_salary = float(candidate['expected_salary'])
                salary_score = 1.0 if job['salary_min'] <= candidate_salary <= job['salary_max'] else 0.0
            except (ValueError, TypeError):
                salary_score = 0.0
            
            # Calculate weighted total score
            total_score = (
                skill_score * weights['skills'] +
                education_score * weights['education'] +
                experience_score * weights['experience'] +
                location_score * weights['location'] +
                salary_score * weights['salary']
            )
            
            # Store results
            results.append({
                'job_title': job['title'],
                'candidate_name': candidate['name'],
                'total_score': total_score,
                'skill_score': skill_score,
                'education_score': education_score,
                'experience_score': experience_score,
                'location_score': location_score,
                'salary_score': salary_score
            })
    
    return pd.DataFrame(results)

# Enhanced Visualization Functions
def create_match_visualizations(match_df, job_title):
    """Create comprehensive visualizations for match results"""
    
    # Filter data for specific job
    job_matches = match_df[match_df['job_title'] == job_title].sort_values('total_score', ascending=False)
    
    # 1. Overall Match Scores Bar Chart
    fig1 = px.bar(
        job_matches,
        x='total_score',
        y='candidate_name',
        title=f'Overall Match Scores for {job_title}',
        labels={'total_score': 'Match Score (%)', 'candidate_name': 'Candidate'},
        orientation='h'
    )
    fig1.update_layout(
        height=400,
        xaxis_range=[0, 100],
        showlegend=False,
        xaxis_title='Match Score (%)',
        yaxis_title='Candidate',
        title_x=0.5
    )
    
    # 2. Detailed Score Breakdown
    score_categories = ['skill_score', 'education_score', 'experience_score', 
                       'location_score', 'salary_score']
    
    fig2 = go.Figure()
    for category in score_categories:
        fig2.add_trace(go.Bar(
            name=category.replace('_', ' ').title(),
            x=job_matches['candidate_name'],
            y=job_matches[category] * 100,
        ))
    
    fig2.update_layout(
        title=f'Detailed Score Breakdown for {job_title}',
        xaxis_title='Candidate',
        yaxis_title='Score (%)',
        barmode='group',
        height=500,
        showlegend=True,
        legend_title='Criteria',
        title_x=0.5
    )
    
    # 3. Radar Chart for Top 3 Candidates
    top_3_candidates = job_matches.head(3)
    
    fig3 = go.Figure()
    for _, candidate in top_3_candidates.iterrows():
        fig3.add_trace(go.Scatterpolar(
            r=[candidate[cat] * 100 for cat in score_categories],
            theta=[cat.replace('_', ' ').title() for cat in score_categories],
            fill='toself',
            name=candidate['candidate_name']
        ))
    
    fig3.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )),
        showlegend=True,
        title=f'Top 3 Candidates Comparison for {job_title}',
        title_x=0.5
    )
    
    return fig1, fig2, fig3

def display_match_results(match_df):
    """Display comprehensive match results"""
    
    for job_title in match_df['job_title'].unique():
        print(f"\n{'='*80}")
        print(f"Match Results for: {job_title}")
        print(f"{'='*80}")
        
        # Get matches for this job
        job_matches = match_df[match_df['job_title'] == job_title].sort_values('total_score', ascending=False)
        
        # Display top 5 candidates
        print("\nTop 5 Candidates:")
        print("-"*80)
        for idx, match in job_matches.head().iterrows():
            print(f"\nRank {idx + 1}: {match['candidate_name']}")
            print(f"Overall Match Score: {match['total_score']*100:.2f}%")
            print(f"Skill Match: {match['skill_score']*100:.2f}%")
            print(f"Education Match: {match['education_score']*100:.2f}%")
            print(f"Experience Match: {match['experience_score']*100:.2f}%")
            print(f"Location Match: {match['location_score']*100:.2f}%")
            print(f"Salary Match: {match['salary_score']*100:.2f}%")
        
        # Create and display visualizations
        fig1, fig2, fig3 = create_match_visualizations(match_df, job_title)
        fig1.show()
        fig2.show()
        fig3.show()

def main():
    """Main execution function"""
    try:
        # Process job data
        jobs_df = pd.DataFrame(jobs_data)
        jobs_df['processed_requirements'] = jobs_df['requirements'].apply(preprocess_text)
        
        # Process candidate data
        processed_candidates = []
        for candidate in candidates_data:
            entities = extract_entities(candidate['resume'])
            candidate_profile = {
                'name': candidate['name'],
                'skills': entities['skills'],
                'education': entities['education'],
                'experience': entities['experience'],
                'location': entities['location'],
                'expected_salary': entities['expected_salary']
            }
            processed_candidates.append(candidate_profile)
        
        candidates_df = pd.DataFrame(processed_candidates)
        
        # Prepare text for Word2Vec
        all_text = jobs_df['processed_requirements'].tolist()
        all_text.extend([candidate['skills'] for candidate in processed_candidates])
        sentences = prepare_sentences(all_text)
        
        # Train Word2Vec model
        w2v_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)
        
        # Compute match scores
        match_df = compute_match_scores(jobs_df, candidates_df, w2v_model)
        
        # Display results and visualizations
        display_match_results(match_df)
        
        # Save results to CSV
        match_df.to_csv('job_matching_results.csv', index=False)
        print("\nResults have been saved to 'job_matching_results.csv'")
        
        return match_df
        
    except Exception as e:
        print(f"An error occurred in main execution: {str(e)}")
        raise

def streamlit_app():
    """Streamlit interface"""
    try:
        st.set_page_config(page_title="Job-Candidate Matching System", layout="wide")
        
        # Add custom CSS
        st.markdown("""
            <style>
                .main {
                    padding: 2rem;
                }
                .stTitle {
                    color: #2c3e50;
                    font-size: 2.5rem;
                    font-weight: 700;
                    margin-bottom: 2rem;
                }
                .stSubheader {
                    color: #34495e;
                    font-size: 1.8rem;
                    font-weight: 600;
                    margin: 1.5rem 0;
                }
            </style>
        """, unsafe_allow_html=True)
        
        st.title("Job-Candidate Matching System")
        
        # Process data
        with st.spinner("Processing job and candidate data..."):
            jobs_df = pd.DataFrame(jobs_data)
            jobs_df['processed_requirements'] = jobs_df['requirements'].apply(preprocess_text)
            
            processed_candidates = []
            for candidate in candidates_data:
                entities = extract_entities(candidate['resume'])
                candidate_profile = {
                    'name': candidate['name'],
                    'skills': entities['skills'],
                    'education': entities['education'],
                    'experience': entities['experience'],
                    'location': entities['location'],
                    'expected_salary': entities['expected_salary']
                }
                processed_candidates.append(candidate_profile)
            
            candidates_df = pd.DataFrame(processed_candidates)
            
            # Prepare text for Word2Vec
            all_text = jobs_df['processed_requirements'].tolist()
            all_text.extend([candidate['skills'] for candidate in processed_candidates])
            sentences = prepare_sentences(all_text)
            
            # Train Word2Vec model
            w2v_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)
            
            # Compute matches
            match_df = compute_match_scores(jobs_df, candidates_df, w2v_model)
        
        # Sidebar for job selection and filters
        st.sidebar.header("Filters")
        
        # Job selection
        job_title = st.sidebar.selectbox(
            "Select Job Position",
            options=jobs_df['title'].unique()
        )
        
        # Score threshold filter
        min_score = st.sidebar.slider(
            "Minimum Match Score",
            min_value=0,
            max_value=100,
            value=50,
            step=5
        )
        
        # Get selected job details
        selected_job = jobs_df[jobs_df['title'] == job_title].iloc[0]
        
        # Main content area
        col1, col2 = st.columns([2, 1])
        
        with col1:
            st.subheader("Job Details")
            st.write(f"**Description:** {selected_job['description']}")
            st.write(f"**Location:** {selected_job['location']}")
            st.write(f"**Salary Range:** ₹{selected_job['salary_min']:,} - ₹{selected_job['salary_max']:,}")
            st.write(f"**Required Education:** {selected_job['education']}")
            st.write(f"**Minimum Experience:** {selected_job['experience_min']} years")
        
        # Filter matches based on minimum score
        job_matches = match_df[
            (match_df['job_title'] == job_title) & 
            (match_df['total_score'] * 100 >= min_score)
        ].sort_values('total_score', ascending=False)
        
        # Display visualizations
        st.subheader("Match Analysis")
        
        fig1, fig2, fig3 = create_match_visualizations(match_df, job_title)
        
        st.plotly_chart(fig1, use_container_width=True)
        st.plotly_chart(fig2, use_container_width=True)
        st.plotly_chart(fig3, use_container_width=True)
        
        # Display detailed results table
        st.subheader("Detailed Match Scores")
        
        # Format the dataframe for display
        display_df = job_matches[[
            'candidate_name', 'total_score', 'skill_score',
            'education_score', 'experience_score', 'location_score', 'salary_score'
        ]].copy()
        
        # Convert scores to percentages
        for col in display_df.columns[1:]:
            display_df[col] = display_df[col] * 100
        
        # Style the dataframe
        styled_df = display_df.style.format({
            'total_score': '{:.1f}%',
            'skill_score': '{:.1f}%',
            'education_score': '{:.1f}%',
            'experience_score': '{:.1f}%',
            'location_score': '{:.1f}%',
            'salary_score': '{:.1f}%'
        }).background_gradient(cmap='YlOrRd', subset=display_df.columns[1:])
        
        st.dataframe(styled_df, use_container_width=True)
        
        # Download results
        if st.button("Download Results"):
            csv = job_matches.to_csv(index=False)
            st.download_button(
                label="Download CSV",
                data=csv,
                file_name=f"job_matches_{job_title.lower().replace(' ', '_')}.csv",
                mime="text/csv"
            )
            
    except Exception as e:
        st.error(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    # Choose between regular execution and Streamlit interface
    import sys
    if len(sys.argv) > 1 and sys.argv[1] == '--streamlit':
        streamlit_app()
    else:
        main()